# Week 5: Embedding Database Optimization

This week’s assignment extends the previous weeks’ work by combining dense-vector semantic search with sparse keyword-based filtering. You will build a **hybrid retrieval system** that stores document chunks along with metadata in a SQLite+FAISS index, and performs both FAISS (vector) and full-text keyword search.  The idea is to “have the best of both worlds”: exact keyword matches (via SQLite FTS5 or BM25) **and** semantic similarity (via FAISS). In practice, hybrid methods (e.g. weighted score fusion or reciprocal rank fusion) can improve result relevance over using vectors or keywords alone.



## 📚 Learning Objectives

* **Hybrid Retrieval Techniques:** Understand how to combine dense vector search (FAISS) with sparse keyword search (SQLite FTS5 or BM25). Learn why pure semantic or pure keyword search alone can miss relevant results, and how hybrid search can address both broad “vibe” matches and exact queries.

* **Metadata and Indexing:** Learn to store document metadata (title, authors, year, keywords, etc.) alongside text chunks and their embeddings. You will design a combined index (SQLite tables + FAISS index) so that each text chunk has associated metadata fields.

* **Hybrid Ranking Strategies:** Explore ranking or fusion strategies to merge vector and keyword scores. For example, you might compute a weighted sum of normalized scores, or use **reciprocal rank fusion (RRF)** as a simple ensemble. The goal is to experiment with different score-combination methods and weightings for the final ranking.

* **Evaluation (Recall/Hit Rate):** Learn to evaluate retrieval quality. You should measure metrics like **recall** or **hit rate** (the proportion of queries where a relevant doc appears in the top-k results). You will compare the effectiveness of vector-only search, keyword-only search, and the hybrid approach on example queries.

## Project Design

You will **extend your Week 4 project** by adding metadata storage and keyword search, then implementing a hybrid retrieval pipeline. Key tasks include:

* **Index Structure:** Build or extend your SQLite + FAISS index to store *document metadata*, text *chunks*, and their *embeddings*. For example, use a SQLite table `documents(doc_id, title, author, year, keywords, ...)` and an FTS5 table (e.g. `doc_chunks`) that indexes the chunk text. The FAISS index should store the corresponding embeddings (one embedding per chunk) keyed by `doc_id`.

* **Keyword Search (FTS5/BM25):** Implement sparse keyword search over the text chunks. The simplest way is to use **SQLite FTS5**: create a virtual FTS table on the chunk text, so that queries like `WHERE doc_chunks MATCH 'term'` return relevant rows. Alternatively, you can use a BM25 library (e.g. Python’s `rank_bm25`) to rank chunks by BM25 similarity. Either way, you should be able to retrieve the top-k chunks by exact keyword relevance.

* **Hybrid Retrieval:** For a given user query, perform **two separate searches**: one in FAISS (semantic search) and one in the keyword index (FTS5 or BM25). Each returns its own top-k results with scores. You will then *merge* these results. For example, you could normalize the FAISS distances and FTS/BM25 scores, then compute a **weighted sum** or use **reciprocal rank fusion (RRF)** to re-rank the combined set.  The final output should be a single ranked list of document chunks or pages.

* **Performance Evaluation:** Design an evaluation to compare methods. For instance, prepare at least **10 test queries** with known relevant documents. Then measure for each method (vector-only, keyword-only, hybrid) how often a relevant document appears in the top-3 (or top-k) results — i.e., the hit rate or recall. Report these metrics (e.g. “Recall\@3”) to see whether hybrid search improves over the baselines.



## Starter Code Snippets

Here are some example code snippets and schemas to help you get started:

* **SQLite schema:** Define a table for document metadata and an FTS5 table for text. For example:

  ```sql
  CREATE TABLE documents (
      doc_id    INTEGER PRIMARY KEY,
      title     TEXT,
      author    TEXT,
      year      INTEGER,
      keywords  TEXT
  );
  CREATE VIRTUAL TABLE doc_chunks USING fts5(
      content,                      -- chunk text
      content='documents',          -- external content table
      content_rowid='doc_id'        -- link to documents.doc_id
  );
  ```

  This creates an FTS5 index on the `content` column (chunk text) referencing the `documents` table.

* **Inserting data:** Insert your documents and chunk text. For example, in Python:

  ```python
  conn = sqlite3.connect("mydata.db")
  # Insert document metadata
  conn.execute("INSERT INTO documents VALUES (?, ?, ?, ?, ?)",
               (doc_id, title, author, year, keywords))
  # Insert chunk text into FTS table, linking by rowid
  conn.execute("INSERT INTO doc_chunks(rowid, content) VALUES (?, ?)",
               (doc_id, chunk_text))
  conn.commit()
  ```

  Or in raw SQL, you might SELECT from a content table into the FTS table as shown in.

* **Keyword query (FTS5):** A full-text query can be written as:

  ```sql
  SELECT doc_id, title
  FROM documents
  JOIN doc_chunks ON documents.doc_id = doc_chunks.rowid
  WHERE doc_chunks MATCH 'search terms'
  LIMIT 5;
  ```

  This returns documents whose chunks match the query terms.

* **BM25 example (Python):** If you use `rank_bm25`, example usage is:

  ```python
  from rank_bm25 import BM25Okapi
  docs = ["text of doc1 ...", "text of doc2 ...", ...]
  tokenized_docs = [doc.split() for doc in docs]
  bm25 = BM25Okapi(tokenized_docs)
  query = "example query"
  tokenized_query = query.split()
  top_docs = bm25.get_top_n(tokenized_query, docs, n=3)
  ```

  This returns the top 3 documents by BM25 score.

* **Hybrid score merging:** Here’s a simple Python example of a weighted-sum merger:

  ```python
  def hybrid_score(vec_score, key_score, alpha=0.5):
      # Assume vec_score and key_score are normalized (0-1).
      return alpha * vec_score + (1 - alpha) * key_score

  # Example usage for re-ranking top results:
  combined = []
  for doc, v_score in faiss_results:
      k_score = keyword_scores.get(doc, 0.0)
      combined_score = hybrid_score(v_score, k_score, alpha=0.6)
      combined.append((doc, combined_score))
  combined.sort(key=lambda x: x[1], reverse=True)
  top_k = combined[:3]
  ```

  You can adjust `alpha` or use other formulas (e.g. max, reciprocal rank fusion).

* **FastAPI route skeleton:** To serve hybrid search via an API, you might write:

  ```python
  from fastapi import FastAPI
  app = FastAPI()

  @app.get("/hybrid_search")
  async def hybrid_search(query: str, k: int = 3):
      # 1. Compute query embedding for FAISS
      # 2. Get top-k from FAISS and top-k from SQLite FTS/BM25
      # 3. Merge scores (as above) and select final top-k documents
      return {"results": top_k_results}
  ```

  This endpoint takes a `query` string and returns the top-k hybrid results in JSON.



## Deliverables

Your submission should include:

* The **updated SQLite+FAISS index** (or code to build it) that contains the document chunks, embeddings, and metadata.
* The **hybrid retrieval pipeline code**, including FAISS search, FTS/BM25 search, and the score-merging logic.
* An **evaluation notebook** (e.g. Jupyter) showing at least 10 example queries and reporting metrics (e.g. recall or hit rate @3) for vector-only, keyword-only, and hybrid search.
* A **FastAPI endpoint** implementation (`/hybrid_search`) that returns the hybrid top-3 results for a given query (as JSON).

Include comments in your code to explain each step. Your evaluation should show whether the hybrid method improves over using vectors or keywords alone.


**References:** This assignment is based on standard practices for hybrid search using FAISS and SQLite FTS5. Reciprocal rank fusion and other fusion methods are known techniques in information retrieval. For more background on hit rate metrics, see analytics tutorials on search evaluation.
